#Install Library

In [ ]:
pip install grad-cam -q

In [ ]:
# Install Timm (Need to restart the runtime after finish install )
!pip install git+https://github.com/rwightman/pytorch-image-models.git
!pip install lightning transformers datasets evaluate pillow==9.2.0

⚠ ☝ Restart runtime

##Load dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!cp /content/drive/MyDrive/Augmentation_Datasets/test.zip .


In [ ]:
!unzip -q '/content/drive/MyDrive/Datasets/test.zip' -d '/content/drive/MyDrive/Datasets/test.zip'

In [ ]:
import torch
import torch.nn as nn
import torchvision.transforms as T
from torch.utils.data import DataLoader, random_split, Dataset

# Pytorch Image model (TIMM) library: a library for state-of-the-art image classification
import timm
import timm.optim
import timm.scheduler
from timm.data import ImageDataset, create_dataset, create_loader
from timm.data.transforms_factory import create_transform

import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import classification_report
from PIL import Image

import evaluate

import numpy as np
import pandas as pd
from scipy import stats

from tqdm.notebook import tqdm

import glob

from sklearn.model_selection import StratifiedKFold,KFold

from lightning.fabric import Fabric

from copy import copy

import shutil

from sklearn.utils.class_weight import compute_class_weight

In [ ]:
try:
    from torchinfo import summary
except:
    print("[INFO] Couldn't find torchinfo... installing it.")
    !pip install -q torchinfo
    from torchinfo import summary

In [ ]:
# Select model (List of available is shown above)
mobilenetv3_large_100 =  timm.create_model('efficientnet_checkpoint_fold0.pt', pretrained=True)

# Modify the model for your number of classes
mobilenetv3_large_100.classifier = nn.Linear(in_features=mobilenetv3_large_100.classifier.in_features, out_features=4)

# Print a summary using torchinfo (uncomment for actual output)
summary(model=mobilenetv3_large_100, # Pass the model instance directly
        input_size=(16, 3, 224, 224), # make sure this is "input_size", not "input_shape"
        # col_names=["input_size"], # uncomment for smaller output
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"]
)

#Grand CAM

In [ ]:
import warnings
warnings.filterwarnings('ignore')
from torchvision import models
import numpy as np
import cv2
import requests
from pytorch_grad_cam import GradCAM
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
from pytorch_grad_cam.utils.image import show_cam_on_image, \
    deprocess_image, \
    preprocess_image
from PIL import Image
import torch
import torchvision.transforms as transforms
import os
from PIL import Image
import matplotlib.pyplot as plt


In [ ]:
import os

source_dir = '/content/test'

img_paths = []

for root , dir, files in os.walk(source_dir):
  for file in files:
    file_path = os.path.join(root,file)
    img_paths.append(file_path)
print(len(img_paths))

In [ ]:
import os
# Define the directory paths
test_ad = '/content/drive/MyDrive/Datasets/3_cls/test/AD'
test_control = '/content/drive/MyDrive/Datasets/3_cls/test/CONTROL'
test_pd = '/content/drive/MyDrive/Datasets/3_cls/test/PD'

train_pd = '/content/drive/MyDrive/Datasets/3_cls/train/PD'
train_ad = '/content/drive/MyDrive/Datasets/3_cls/train/AD'
train_control = '/content/drive/MyDrive/Datasets/3_cls/train/CONTROL'

print("Test Ad: ",len(os.listdir(test_ad)))
print("Train Ad: ",len(os.listdir(train_ad)))
print("Test pd: ",len(os.listdir(test_pd)))
print("Train pd: ",len(os.listdir(train_pd)))
print("Train control: ",len(os.listdir(train_control)))
print("Test control: ",len(os.listdir(test_control)))


In [ ]:
model_path = '/content/efficientnet_checkpoint_fold0.pt'

In [ ]:
# ใช้ timm เพื่อสร้างโมเดลและโหลดพารามิเตอร์ pretrained
model = timm.create_model('tf_efficientnetv2_b0.in1k', pretrained=True)
model.eval()


In [ ]:
import numpy as np
import cv2
from PIL import Image
from pytorch_grad_cam import GradCAM
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
from pytorch_grad_cam.utils.image import show_cam_on_image, preprocess_image
import timm
import matplotlib.pyplot as plt

# โหลดโมเดล
model = timm.create_model('tf_efficientnetv2_b0.in1k', pretrained=True)
model.eval()

# โหลดและเตรียมภาพ
image_url = "/content/test/AD/AD_2680.png"
img = np.array(Image.open(image_url))
img = cv2.resize(img, (224, 224))

# เช็คว่าภาพเป็น grayscale หรือไม่ ถ้าใช่ให้แปลงเป็น RGB
if img.ndim == 2:  # Grayscale image
    img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)

img = np.float32(img) / 255
input_tensor = preprocess_image(img, mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

print(input_tensor)
print(input_tensor.shape)

# กำหนด target สำหรับ Grad-CAM
targets = [ClassifierOutputTarget(0)]  # เลือกคลาสที่คุณสนใจ เช่น 0

# เลือก target layer ที่เหมาะสมจากโมเดล
target_layers = [model.blocks[-1]]  # ตัวอย่างการเลือกเลเยอร์ที่เหมาะสม

# สร้าง Grad-CAM
with GradCAM(model=model, target_layers=target_layers) as cam:
    grayscale_cams = cam(input_tensor=input_tensor, targets=targets)
    print(grayscale_cams)

    # แปลงเป็นภาพ RGB และแสดงผล
    cam_image = show_cam_on_image(img, grayscale_cams[0, :], use_rgb=True)

# แปลง grayscale_cams เป็นภาพ RGB
cam = np.uint8(255 * grayscale_cams[0, :])
cam = cv2.merge([cam, cam, cam])

# ผสานภาพต้นฉบับและ CAM เข้าด้วยกัน
images = np.hstack((np.uint8(255 * img), cam, cam_image))

# แสดงภาพ
fig, axs = plt.subplots(1, 3, figsize=(15, 5))
axs[0].imshow(np.uint8(255 * img))
axs[0].set_title('Original Image')
axs[0].axis('off')

axs[1].imshow(cam)
axs[1].set_title('Grad-CAM')
axs[1].axis('off')

axs[2].imshow(cam_image)
axs[2].set_title('Overlay Image')
axs[2].axis('off')

plt.show()


In [ ]:
import os
import torch
from PIL import Image
import numpy as np
import cv2
import timm  # Import timm to load the model directly
from pytorch_grad_cam import GradCAM
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
from pytorch_grad_cam.utils.image import show_cam_on_image, preprocess_image

# Load the model directly from timm
model = timm.create_model('tf_efficientnetv2_b0.in1k', pretrained=True)
model = model.eval()
model = model.to('cpu')  # Move the model to CPU if needed

# ... rest of your code (from cell 53)
image_url = "/content/test/AD/AD_2680.png"
img = np.array(Image.open(image_url))
# ... (continue with the rest of your code)

In [ ]:
def grand_cam(img_path, model, class_idx):
    # Load image and apply transforms
    img = np.array(Image.open(img_path))
    img = cv2.resize(img, (224, 224))

    # Check if the image is grayscale and convert to RGB if necessary
    if img.ndim == 2:  # Grayscale image
        img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)

    img = np.float32(img) / 255
    input_tensor = preprocess_image(img, mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

    # Grad-CAM Process
    targets = [ClassifierOutputTarget(class_idx)]
    target_layers = [model.blocks[-1]]  # Using the last block of EfficientNetV2

    with GradCAM(model=model, target_layers=target_layers) as cam:
        grayscale_cams = cam(input_tensor=input_tensor, targets=targets)
        cam_image = show_cam_on_image(img, grayscale_cams[0, :], use_rgb=True)

    cam = np.uint8(255 * grayscale_cams[0, :])
    cam = cv2.merge([cam, cam, cam])
    images = np.hstack((np.uint8(255 * img), cam, cam_image))
    pil_image = Image.fromarray(images)
    print(f"Image : {os.path.basename(img_path)}")
    plt.imshow(pil_image)
    plt.pause(0.1)

In [ ]:
# Call the function with the model and image path
grand_cam('/content/test/AD/AD_2680.png', model, 0)

In [ ]:
def grand_cam(img_path, model, class_idx):
    # Load image and apply transforms
    img = np.array(Image.open(img_path))
    img = cv2.resize(img, (224, 224))

    # Check if the image is grayscale and convert to RGB if necessary
    if img.ndim == 2:  # Grayscale image
        img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)

    img = np.float32(img) / 255
    input_tensor = preprocess_image(img, mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

    # Grad-CAM Process
    targets = [ClassifierOutputTarget(class_idx)]
    target_layers = [model.blocks[-1]]  # Using the last block of EfficientNetV2

    with GradCAM(model=model, target_layers=target_layers) as cam:
        grayscale_cams = cam(input_tensor=input_tensor, targets=targets)
        cam_image = show_cam_on_image(img, grayscale_cams[0, :], use_rgb=True)

    cam = np.uint8(255 * grayscale_cams[0, :])
    cam = cv2.merge([cam, cam, cam])
    images = np.hstack((np.uint8(255 * img), cam, cam_image))
    pil_image = Image.fromarray(images)
    print(f"Image : {os.path.basename(img_path)}")
    plt.imshow(pil_image)
    plt.pause(0.1)

#AD

In [ ]:
# Directory path and corresponding class index
image_directory = '/content/test/AD'
image_paths = [os.path.join(image_directory, filename) for filename in os.listdir(image_directory) if filename.endswith('.png')]

for img_path in image_paths:
  grand_cam(img_path,model,0)

#CONTROL

In [ ]:
# Directory path and corresponding class index
image_directory = '/content/test/CONTROL'
image_paths = [os.path.join(image_directory, filename) for filename in os.listdir(image_directory) if filename.endswith('.png')]

for img_path in image_paths:
  grand_cam(img_path,model,1)


#PD

In [ ]:
image_directory = '/content/test/PD'
image_paths = [os.path.join(image_directory, filename) for filename in os.listdir(image_directory) if filename.endswith('.png')]

for img_path in image_paths:
  grand_cam(img_path,model,2)

In [ ]:
def grand_cam(img_path, model, class_idx):
    # Load image and apply transforms
    img = np.array(Image.open(img_path))
    img = cv2.resize(img, (224, 224))

    # Check if the image is grayscale and convert to RGB if necessary
    if img.ndim == 2:  # Grayscale image
        img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)

    img = np.float32(img) / 255
    input_tensor = preprocess_image(img, mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

    # Grad-CAM Process
    targets = [ClassifierOutputTarget(class_idx)]
    target_layers = [model.blocks[-1]]  # Using the last block of EfficientNetV2

    with GradCAM(model=model, target_layers=target_layers) as cam:
        grayscale_cams = cam(input_tensor=input_tensor, targets=targets)
        cam_image = show_cam_on_image(img, grayscale_cams[0, :], use_rgb=True)

    cam = np.uint8(255 * grayscale_cams[0, :])
    cam = cv2.merge([cam, cam, cam])
    images = np.hstack((np.uint8(255 * img), cam, cam_image))
    pil_image = Image.fromarray(images)
    print(f"Image : {os.path.basename(img_path)}")
    plt.imshow(pil_image)
    plt.pause(0.1)

In [ ]:
img ='/content/test/PD/PD_308.png'
grand_cam(img, model, 3) # Added img as the first argument

In [ ]:
image_directory = '/content/test/AD'
image_paths = [os.path.join(image_directory, filename) for filename in os.listdir(image_directory) if filename.endswith('.png')]

for img_path in image_paths:
  grand_cam(img_path,model,0)

In [ ]:
image_directory = '/content/test/CONTROL'
image_paths = [os.path.join(image_directory, filename) for filename in os.listdir(image_directory) if filename.endswith('.png')]

for img_path in image_paths:
  grand_cam(img_path,model,1)

In [ ]:
image_directory = '/content/test/PD'
image_paths = [os.path.join(image_directory, filename) for filename in os.listdir(image_directory) if filename.endswith('.png')]

for img_path in image_paths:
  grand_cam(img_path,model,2)

In [ ]:
import torchvision.transforms as transforms

# กำหนด transforms สำหรับการทำ Data Augmentation
data_augmentation = transforms.Compose([
    transforms.RandomResizedCrop(224),  # ครอบตัดภาพให้มีขนาด 224x224 แบบสุ่ม
    transforms.RandomHorizontalFlip(),  # กลับด้านภาพแนวนอนแบบสุ่ม
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),  # ปรับค่าความสว่าง สีสัน ฯลฯ
    transforms.ToTensor(),  # แปลงภาพให้เป็น Tensor
    # Removed the Lambda function that repeats the grayscale image across 3 channels
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # ปรับค่า Normalize
])

# โหลดและเตรียมภาพพร้อมการทำ Data Augmentation
image = Image.open(image_url).convert('RGB') # Ensure image is in RGB format
image_augmented = data_augmentation(image)

# แปลง Tensor กลับเป็นรูปภาพสำหรับการแสดงผล
image_augmented = image_augmented.permute(1, 2, 0).numpy()

In [ ]:
import matplotlib.pyplot as plt

# โหลดและเตรียมภาพพร้อมการทำ Data Augmentation
image = Image.open(image_url).convert('RGB')  # Ensure image is in RGB format
image_augmented = data_augmentation(image)

# แปลง Tensor กลับเป็นรูปภาพสำหรับการแสดงผล
image_augmented = image_augmented.permute(1, 2, 0).numpy()

# เนื่องจากภาพถูก Normalize ให้มีค่าอยู่ระหว่าง 0 ถึง 1 เราจึงต้องปรับค่ากลับมาอยู่ในช่วง 0 ถึง 1
image_augmented = (image_augmented - image_augmented.min()) / (image_augmented.max() - image_augmented.min())

# แสดงผลภาพ
plt.imshow(image_augmented)
plt.axis('off')  # ปิดการแสดงแกน x และ y
plt.title("Augmented Image")
plt.show()


In [ ]:
import os
import matplotlib.pyplot as plt
from PIL import Image
import torchvision.transforms as transforms

# กำหนด transforms สำหรับการทำ Data Augmentation
data_augmentation = transforms.Compose([
    transforms.RandomResizedCrop(224),  # ครอบตัดภาพให้มีขนาด 224x224 แบบสุ่ม
    transforms.RandomHorizontalFlip(),  # กลับด้านภาพแนวนอนแบบสุ่ม
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),  # ปรับค่าความสว่าง สีสัน ฯลฯ
    transforms.ToTensor(),  # แปลงภาพให้เป็น Tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # ปรับค่า Normalize
])

# ฟังก์ชันในการโหลดและแสดงผลภาพ
def show_augmented_image(image_path, title):
    image = Image.open(image_path).convert('RGB')  # Ensure image is in RGB format
    image_augmented = data_augmentation(image)
    image_augmented = image_augmented.permute(1, 2, 0).numpy()
    image_augmented = (image_augmented - image_augmented.min()) / (image_augmented.max() - image_augmented.min())

    plt.imshow(image_augmented)
    plt.axis('off')
    plt.title(title)
    plt.show()

# Directory path และ corresponding class index
image_directories = {
    "AD": '/content/test/AD',
    "PD": '/content/test/PD',
    "CONTROL": '/content/test/CONTROL'
}

# แสดงผลภาพจากแต่ละหมวดหมู่
for category, directory in image_directories.items():
    image_paths = [os.path.join(directory, filename) for filename in os.listdir(directory) if filename.endswith('.png')]
    if image_paths:
        show_augmented_image(image_paths[0], category)


In [ ]:
import os
import matplotlib.pyplot as plt
from PIL import Image
import torchvision.transforms as transforms

# Define transforms for specific augmentations
rotation_45 = transforms.Compose([
    transforms.RandomRotation(degrees=45),
    transforms.ToTensor()
])

rotation_135 = transforms.Compose([
    transforms.RandomRotation(degrees=135),
    transforms.ToTensor()
])

zoom_in = transforms.Compose([
    transforms.Resize((int(224 * 1.1), int(224 * 1.1))),  # 10% Zoom
    transforms.CenterCrop(224),
    transforms.ToTensor()
])

# Function to load and show the augmented images
def show_augmented_images(image_path, title):
    image = Image.open(image_path).convert('RGB')  # Ensure image is in RGB format

    # Apply augmentations
    image_45 = rotation_45(image)
    image_135 = rotation_135(image)
    image_zoom = zoom_in(image)

    # Convert tensors to numpy arrays for displaying
    image_45 = image_45.permute(1, 2, 0).numpy()
    image_135 = image_135.permute(1, 2, 0).numpy()
    image_zoom = image_zoom.permute(1, 2, 0).numpy()

    # Normalize the images for displaying
    images = [image_45, image_135, image_zoom]
    images = [(img - img.min()) / (img.max() - img.min()) for img in images]

    # Plotting the images
    fig, axes = plt.subplots(1, 3, figsize=(12, 4))
    for ax, img, label in zip(axes, images, ['Rotation 45°', 'Rotation 135°', 'Zoom 10%']):
        ax.imshow(img)
        ax.axis('off')
        ax.set_title(f'{title} - {label}')
    plt.show()

# Directory path and corresponding class index
image_directories = {
    "AD": '/content/test/AD',
    "PD": '/content/test/PD',
    "CONTROL": '/content/test/CONTROL'
}

# Display images from each category
for category, directory in image_directories.items():
    image_paths = [os.path.join(directory, filename) for filename in os.listdir(directory) if filename.endswith('.png')]
    if image_paths:
        show_augmented_images(image_paths[0], category)


In [ ]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import numpy as np

# สมมติให้ y_true คือข้อมูล label จริง และ y_pred คือผลลัพธ์ที่โมเดลทำนายได้
# y_true และ y_pred จะเป็น array ที่มีค่าเป็น label (AD, CONTROL, PD) ของข้อมูลแต่ละตัวอย่าง

# ตัวอย่างของ y_true และ y_pred (คุณต้องแทนที่ด้วยค่าจริงที่ได้จากโมเดลของคุณ)
# Replace ellipsis with actual label values
y_true = np.array(['AD', 'CONTROL', 'PD', 'CONTROL', 'AD', 'PD', 'AD'])
y_pred = np.array(['AD', 'AD', 'PD', 'CONTROL', 'CONTROL', 'PD', 'CONTROL'])

# คำนวณค่า confusion matrix
cm = confusion_matrix(y_true, y_pred, labels=['AD', 'CONTROL', 'PD'])

# คำนวณค่า Precision, Recall, F1 Score, และ Accuracy
report = classification_report(y_true, y_pred, target_names=['AD', 'CONTROL', 'PD'])
accuracy = accuracy_score(y_true, y_pred)

# แสดงผลลัพธ์
print("Confusion Matrix:")
print(cm)
print("\nClassification Report:")
print(report)
print(f"\nAccuracy: {accuracy:.2f}")

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
from sklearn.metrics import classification_report, accuracy_score
import numpy as np
import timm

# กำหนด transforms สำหรับการทำ Data Augmentation
data_transforms = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# โหลด dataset
dataset = datasets.ImageFolder(root='/content/test', transform=data_transforms)

# แบ่ง dataset เป็น train และ test
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

# สร้าง DataLoader สำหรับ train และ test
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# เลือกโมเดลที่จะใช้ (ในที่นี้ใช้ tf_efficientnetv2_b0.in1k จาก timm)
model = timm.create_model('tf_efficientnetv2_b0.in1k', pretrained=True)
model.classifier = nn.Linear(model.classifier.in_features, 3)  # กำหนด output ให้เท่ากับจำนวน class ที่มี

# กำหนด loss function และ optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# การฝึกโมเดล
num_epochs = 10
model.train()

for epoch in range(num_epochs):
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}")

# การทดสอบโมเดล
model.eval()
y_true = []
y_pred = []

with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        y_true.extend(labels.numpy())
        y_pred.extend(preds.numpy())

# คำนวณค่า Precision, Recall, F1 Score, และ Accuracy
y_true = np.array(y_true)
y_pred = np.array(y_pred)

print("Classification Report:")
print(classification_report(y_true, y_pred, target_names=dataset.classes))

accuracy = accuracy_score(y_true, y_pred)
print(f"Accuracy: {accuracy:.2f}")
